In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [119]:
df[df['press'] == 'The Washington Post']['url'].iloc[800]

'https://www.washingtonpost.com/world/2023/02/14/china-united-states-marshall-islands-rongelap/'

In [120]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            # First, try to find elements with the '.article-comm div' selector
            article_elements = soup.select('.mb-xxl-ns')
            
            
            # # # # If not found, try the '.p-m-15' selector
            # if not article_elements:
            #     article_elements = soup.select('.entry-content')

            # # If still not found, try the '.article-text' selector
            # if not article_elements:
            #     article_elements = soup.select('.p-m-15')

            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''
            # print(article_elements)

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [121]:
TheWashingtonPost = df[df['press'] == 'The Washington Post']
TheWashingtonPost

,source,author,title,description,url,urlToImage,publishedAt,content,press
411,"{'id': 'the-washington-post', 'name': 'The Was...",Michelle Singletary,Your spending problem is all in your head,Many people are bad at making good financial d...,https://www.washingtonpost.com/business/get-th...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-11T08:24:08Z,People often ask me what the best budgeting to...,The Washington Post
1523,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Extradition hearing for Russian cybercrime sus...,Greece’s Supreme Court on Wednesday postponed ...,https://www.washingtonpost.com/world/europe/ex...,https://www.washingtonpost.com/pbox.php?url=ht...,2017-11-15T11:54:11Z,"ATHENS, Greece — Greece’s Supreme Court on Wed...",The Washington Post
3484,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,HBO data theft traced to Iranian military hack...,Washington Post HBO data theft traced to Irani...,https://www.washingtonpost.com/world/national-...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-21T16:59:38Z,Federal prosecutors announced charges Tuesday ...,The Washington Post
3579,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Why BuzzFeed teamed with a far-right figure to...,Newsweek Why BuzzFeed teamed with a far-right ...,https://www.washingtonpost.com/lifestyle/style...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-21T21:11:17Z,Journalism can make for strange bedfellows. Ca...,The Washington Post
3745,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Tech Giants Can't Beat 25000 Mongolian Bitcoin...,Tech Giants Can't Beat 25000 Mongolian Bitcoin...,https://www.washingtonpost.com/business/tech-g...,https://www.washingtonpost.com/pbox.php?url=ht...,2017-11-22T09:27:15Z,"Bitcoin pushed beyond $8,000 this week, a boon...",The Washington Post
...,...,...,...,...,...,...,...,...,...
727475,"{'id': 'the-washington-post', 'name': 'The Was...",bookofjoe,Men charged by U.S. with laundering more than ...,Article URL: https://www.washingtonpost.com/na...,https://www.washingtonpost.com/national-securi...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2023-08-24T22:58:25Z,Comment on this story\r\nComment\r\nFederal au...,The Washington Post
731688,"{'id': 'the-washington-post', 'name': 'The Was...",Jeff Stein,U.S. debt is growing as possible government sh...,U.S. debt is growing as possible government sh...,https://www.washingtonpost.com/business/2023/0...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2023-09-03T10:06:29Z,Comment on this story\r\nComment\r\nThe federa...,The Washington Post
732642,"{'id': 'the-washington-post', 'name': 'The Was...",Shira Ovide,Advice to use ChatGPT like a pro,Chatbots can be amazingly helpful — but only i...,https://www.washingtonpost.com/technology/2023...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2023-09-05T17:12:21Z,Comment on this story\r\nComment\r\nThis artic...,The Washington Post
735427,"{'id': 'the-washington-post', 'name': 'The Was...",Tory Newmyer,Financial fraudsters should be listed in a nat...,CFTC Commissioner Christy Goldsmith Romero say...,https://www.washingtonpost.com/business/2023/0...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2023-09-11T22:08:23Z,Comment on this story\r\nComment\r\nA top fina...,The Washington Post


In [62]:

url = 'https://www.fastcompany.com/90602476/tesla-invests-1-5-billion-in-bitcoin-sending-the-price-soaring-to-new-heights?icid=dan902:754:0:editRecirc'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, "html.parser")
print(response.status_code)
# soup

403


In [26]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents


In [122]:
head = TheWashingtonPost.head()
head['content_full'] = get_contents(head)
head

100%|██████████| 5/5 [00:53<00:00, 10.73s/it]
C:\Users\boyu571\AppData\Local\Temp\ipykernel_5172\480069068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
411,"{'id': 'the-washington-post', 'name': 'The Was...",Michelle Singletary,Your spending problem is all in your head,Many people are bad at making good financial d...,https://www.washingtonpost.com/business/get-th...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-11T08:24:08Z,People often ask me what the best budgeting to...,The Washington Post,The solution to your money troubles is to thin...
1523,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Extradition hearing for Russian cybercrime sus...,Greece’s Supreme Court on Wednesday postponed ...,https://www.washingtonpost.com/world/europe/ex...,https://www.washingtonpost.com/pbox.php?url=ht...,2017-11-15T11:54:11Z,"ATHENS, Greece — Greece’s Supreme Court on Wed...",The Washington Post,
3484,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,HBO data theft traced to Iranian military hack...,Washington Post HBO data theft traced to Irani...,https://www.washingtonpost.com/world/national-...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-21T16:59:38Z,Federal prosecutors announced charges Tuesday ...,The Washington Post,"A scene from Season 5, Episode 8 of the HBO se..."
3579,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Why BuzzFeed teamed with a far-right figure to...,Newsweek Why BuzzFeed teamed with a far-right ...,https://www.washingtonpost.com/lifestyle/style...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-21T21:11:17Z,Journalism can make for strange bedfellows. Ca...,The Washington Post,Rep. John Conyers (D-Mich.) settled a complain...
3745,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Tech Giants Can't Beat 25000 Mongolian Bitcoin...,Tech Giants Can't Beat 25000 Mongolian Bitcoin...,https://www.washingtonpost.com/business/tech-g...,https://www.washingtonpost.com/pbox.php?url=ht...,2017-11-22T09:27:15Z,"Bitcoin pushed beyond $8,000 this week, a boon...",The Washington Post,


In [127]:
# Adding the content to the DataFrame
TheWashingtonPost['content_full'] = get_contents(TheWashingtonPost)
TheWashingtonPost

100%|██████████| 849/849 [2:36:54<00:00, 11.09s/it]  
C:\Users\boyu571\AppData\Local\Temp\ipykernel_5172\191520244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TheWashingtonPost['content_full'] = get_contents(TheWashingtonPost)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
411,"{'id': 'the-washington-post', 'name': 'The Was...",Michelle Singletary,Your spending problem is all in your head,Many people are bad at making good financial d...,https://www.washingtonpost.com/business/get-th...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-11T08:24:08Z,People often ask me what the best budgeting to...,The Washington Post,The solution to your money troubles is to thin...
1523,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Extradition hearing for Russian cybercrime sus...,Greece’s Supreme Court on Wednesday postponed ...,https://www.washingtonpost.com/world/europe/ex...,https://www.washingtonpost.com/pbox.php?url=ht...,2017-11-15T11:54:11Z,"ATHENS, Greece — Greece’s Supreme Court on Wed...",The Washington Post,
3484,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,HBO data theft traced to Iranian military hack...,Washington Post HBO data theft traced to Irani...,https://www.washingtonpost.com/world/national-...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-21T16:59:38Z,Federal prosecutors announced charges Tuesday ...,The Washington Post,"A scene from Season 5, Episode 8 of the HBO se..."
3579,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Why BuzzFeed teamed with a far-right figure to...,Newsweek Why BuzzFeed teamed with a far-right ...,https://www.washingtonpost.com/lifestyle/style...,https://www.washingtonpost.com/rf/image_1484w/...,2017-11-21T21:11:17Z,Journalism can make for strange bedfellows. Ca...,The Washington Post,Rep. John Conyers (D-Mich.) settled a complain...
3745,"{'id': 'the-washington-post', 'name': 'The Was...",NaN,Tech Giants Can't Beat 25000 Mongolian Bitcoin...,Tech Giants Can't Beat 25000 Mongolian Bitcoin...,https://www.washingtonpost.com/business/tech-g...,https://www.washingtonpost.com/pbox.php?url=ht...,2017-11-22T09:27:15Z,"Bitcoin pushed beyond $8,000 this week, a boon...",The Washington Post,
...,...,...,...,...,...,...,...,...,...,...
727475,"{'id': 'the-washington-post', 'name': 'The Was...",bookofjoe,Men charged by U.S. with laundering more than ...,Article URL: https://www.washingtonpost.com/na...,https://www.washingtonpost.com/national-securi...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2023-08-24T22:58:25Z,Comment on this story\r\nComment\r\nFederal au...,The Washington Post,FBI Director Christopher A. Wray testifies at ...
731688,"{'id': 'the-washington-post', 'name': 'The Was...",Jeff Stein,U.S. debt is growing as possible government sh...,U.S. debt is growing as possible government sh...,https://www.washingtonpost.com/business/2023/0...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2023-09-03T10:06:29Z,Comment on this story\r\nComment\r\nThe federa...,The Washington Post,Morning light inside the Capitol Rotunda. Cong...
732642,"{'id': 'the-washington-post', 'name': 'The Was...",Shira Ovide,Advice to use ChatGPT like a pro,Chatbots can be amazingly helpful — but only i...,https://www.washingtonpost.com/technology/2023...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2023-09-05T17:12:21Z,Comment on this story\r\nComment\r\nThis artic...,The Washington Post,"My colleagues spent many months trying, failin..."
735427,"{'id': 'the-washington-post', 'name': 'The Was...",Tory Newmyer,Financial fraudsters should be listed in a nat...,CFTC Commissioner Christy Goldsmith Romero say...,https://www.washingtonpost.com/business/2023/0...,https://www.washingtonpost.com/wp-apps/imrs.ph...,2023-09-11T22:08:23Z,Comment on this story\r\nComment\r\nA top fina...,The Washington Post,"Christy Goldsmith Romero, who serves on the Co..."


In [126]:
head['url'].iloc[4]

'https://www.washingtonpost.com/business/tech-giants-cant-beat-25000-mongolian-bitcoin-machines-gadfly/2017/11/22/0a90591e-cf67-11e7-a87b-47f14b73162a_story.html'

In [123]:
head['content_full'].iloc[0]

'The solution to your money troubles is to think more. You’ve got to delve deeper into the mind games being played against you and guard against the things you tell yourself that lead to money mistakes and your financial mismanagement. (iStock)ShareComment on this storyCommentAdd to your saved storiesSavePeople often ask me what the best budgeting tool is to help them better track and control their spending.I always pause, hoping the person is ready for an answer that’s different from what he or she asked.If you have a personal finance question for Washington Post columnist Michelle Singletary, please call 1-855-ASK-POST (1-855-275-7678)ArrowRightA number of websites and mobile apps can turn you into a budgeting wizard, doing everything from downloading your banking information and allowing you to create impressive charts to alerting you when you’ve blown past your self-imposed spending limits.But after years of working one-on-one with financially challenged people, here’s what I’ve fo

In [128]:
empty_rows = TheWashingtonPost[TheWashingtonPost['content_full'] == ""]
print(len(empty_rows))

129


In [16]:
Itsecuritynews['content_full'] = Itsecuritynews['content_full'].replace(
    '\nSorry. We can\'t seem to find the page you\'re looking for.\n', ''
)

C:\Users\boyu571\AppData\Local\Temp\ipykernel_5172\3103017377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Itsecuritynews['content_full'] = Itsecuritynews['content_full'].replace(


In [137]:
empty_rows['url'].iloc[70]

'https://www.washingtonpost.com/business/technology/un-report-north-korea-cyber-experts-raised-up-to-2-billion/2019/08/05/260af7a8-b7eb-11e9-8e83-4e6687e99814_story.html'

<!-- # empty_rows['url'].iloc[0] -->

In [138]:
TheWashingtonPost.to_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\TheWashingtonPost_112.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content